### Patterns in general
- "Each pattern describes a problem which occuurs over and over in our environment, and then describes the core of the solution to that problem, in such way that you can use this solution a million times over, without ever doing it the same way twice" [Alexander et al, "A Pattern Language"]
- Design (and thus patterns) is not independent from the implementation's technology.

### Classic SW DPs
- not data structures, nor algorithms
- not domain-specific architectures for entire subsystems

### DP write-up components
- NAME, context, problem
- KNOWN USES (KU): DP are discovered, not invented
- results, rationale, related DPs
- forces, solution, examples
- DP: description > prescription
- formal fixed schema not a must (but helpful)

## Categories of SQ DP
- __Creational__: concern the ways and means of object instantiation

- __Structural__: deal with the mutual composition of classes or objects

- __Behavioral__: analyze the ways in which classes or objects interact and distribute responsibilities among them

## Prolegomena
- Program to an interface, not to an implementation
- Favor object composition over class inheritance

    > - Strong Coupling -> inherit only when it's really convenient
    > - Not -> hold or wrap

## Hold & Wrap
- Hold: object O has subobject S as an attribute (maybe property)
    - self.S.method or O.S.method
    - Good: Simple, direct, immediate
    - Bad: Strong coupling, often on wrong axis
- Wrap: hold (via private name) plus deligation
    - O.method
    - Good: 
        - explicit (def method(self,...) <- self.S.method)
        - Automatic (delegation in `__getattr__`)
        - Gets coupling right (law of Demeter)\
- Inheritance cannot restrict

Python2: `raise E, V, T`
Python3: `raise E(V).with_traceback(T)`

In [4]:
class RestrictingWrapper(object):
    def __init__(self, wrap, block):
        self._wrap = wrap
        self._block = block
    def __getattr__(self, n):
        if n in self._block:
            raise AttributeError(n)
        return getattr(self._wrap, n)

# Creational Patterns
- not very common in Python because "factory" is essentially built-in

## Problem: We want just one instance to exist
1. \>90% times, use a module instead of a class
    - X: no subclassing, no special methods (`__add__`)...
2. Make just 1 instance (no enforcement)
3. Singleton
    - X: subclassing not really smooth
4. Monostate
    - X: Guido dislikes it
    
### Singleton
```Python
class Singleton(object):
    def __new__(cls, *a, **k):
        if not hasattr(cls, '_inst'):
            cls._inst = super(Singleton, cls).__new__(cls, *a, **k)
        return cls._inst
```
__CONS__: subclassing is a problem:
```Python
class Foo(Singleton): pass
class Bar(Foo): pass
f = Foo(); b = Bar(); # ...???...
```

### Monostate (Borg)
Everything will share the same state
```Python
class Borg(object):
    _shared_state = {}
    def __new__(cls, *a, **k):
        obj = super(Borg, cls).__new__(cls, *a, **k)
        obj.__dict__ = cls._shared_state
        return obj
```
subclassing:
```Python
class Foo(Borg): pass
class Bar(Foo): pass
class Baz(Foo): _shared_state = {}
```
_Data Overriding_ 

## Problem: Not commit to instantiating a specific concrete class
1. Dependency injection
    - no creation; injected from "outside"
    - X: Don't know how many widgets are needed / don't know how many dependencies to inject at the first place
2. Factory subcategory of DPs
    - may create whatever is approperate / reuse existing
    - factory functions (& other callables)
    - factory methods (overridable)
    - abstract factory classes
    
### Factories in Python
Each type/class is intrinsically a factory. Class is callable
- internally, may have `__new__` which can do anything
- externally, it's a callable, interchangeable with any other
- maybe injected directly (no need for boilerplate factory functions)

Modules can be kinda "abstract" factories without inheritance (e.g., `os` can be `posix` or `nt`)

#### Known Use: `type.__call__`: "two-phase construction: new-init"
```Python
def __call__(cls, *a, *k):
    nu = cls.__new__(cls, *a, **k)
    if isinstance(nu, cls):
        cls.__init__(nu, *a, **k)
    return nu
```

#### Factory-function example
```Python
def load(pkg, obj):
    m = __import__(pkg, {}, {}, [obj])
    return getattr(m, obj)
cls = load('p1.p2.p3', 'c4')
```

# Structural Patterns
Masquerading / Adaptation

- __Adapter__: Tweak an interface (both class and object variants)
- __Facade__: Simplify a subsystem's interface
- __Bridge__: let many implementations of an abstraction us many implementation of a functionality (without repetitive coding)
- __Decorator__: reuse + tweak without inheritance
- __Proxy__: decouple from access / location

## Adapter
- client code $\gamma$ requires a protocol $C$
- supplier code $\sigma$ provides different protocol $S$
    - with a superset of $C$'s functionality
- write adapter code $\alpha$ "sneaks in the middle"
    - to $\gamma$, $\alpha$ is a supplier (produces protocol $C$)
    - to $\sigma$, $\alpha$ is a client (consumes protocol $S$)
    - "inside", $\alpha$ implements $C$ (by means of appropriate calls to $S$ on $\sigma$)

In [10]:
## Toy example
# C requires foobar(foo, bar)
# S supplies method barfoo(bar, foo)

# Sigma
class Barfooer(object):
    def barfoo(self, bar, foo):
        pass

# Adapter at Object level
# per-instance, with wrapping delegation
class FoobarWrapper(object):
    def __init__(self, wrappee):
        self.w = wrappee
    def foobar(self, foo, bar):
        return self.w.barfoo(bar, foo)
barfooer = Barfooer()
foobarer = FoobarWrapper(barfooer)

# Adapter at Class level
# per-class, with subclasing and self-delegation
# maybe faster but object level is cleaner
class Foobarer(Barfooer):
    def foobar(self, foo, bar):
        return self.barfoo(bar, foo)

foobarer = Foobarer()

### Adapter KU
- `socket._fileobject`: from sockets to file-like objects
    - socket: unbuffered, file: buffered
    - with much code for buffering
- `doctest.DocTestSuite`: adapts doctest tests to `unittest.TestSuite`
- `dbhash`: adapt dsbdb to dbm
- `StringIO`: adapt `str` or `unicode` to file-like
- `shelve`: adapt "limited dict" (str keys and values, basic methods) to complete mapping
    - via `pickle` for anything from/to string
    - `UserDict.DictMixin` to add other dict methods
    
### Observations: Adapter
- Some real life adapters may require a lot more code
- mixin classes are a great way to help adapt to rich protocols
    - can implement advanced methods on top of fundamental ones
- Adapter occurs at all levels of complexity
- in Python, it's NOT just about classes and their instances (callables) 